# Generate Michigan Road Network

In [40]:
import geopandas as gp
import igraph as ig

## Read in data

First let's read in road segments shapefile. You can find it at [Michigan GIS Open Data](http://gis-michigan.opendata.arcgis.com/datasets/all-roads-v17a).

In [41]:
file1 = 'All_Roads_v17a.geojson'
roads = gp.read_file(file1)
len(roads)

795003

In [45]:
roads.head()

,OBJECTID,FCC,RDNAME,FRADDL,TOADDL,FRADDR,TOADDR,ZIPL,ZIPR,FEDIRP,...,EMP,BPT,EPT,LRS_LINK,LENGTH,VER,ShapeSTLength,ADJ_CNTY,LEGALSYSTEM,geometry
0,1,A41,North Point Rd,11448,11000,11449,11001,49950,49950,,...,0.835,42001455,42001458,34205854200145542001458,1343.174725,17a,1343.174920,0,0,LINESTRING (-88.02735862891302 47.474893562611...
1,2,A41,Harbor Coast Ln,0,0,0,0,0,0,,...,0.657,83021251,83021252,54954408302125183021252,1058.015634,17a,1058.015729,0,0,LINESTRING (-87.90453443038533 47.475922960203...
2,3,A21,M 26,11469,11511,11468,11510,49950,49950,,...,2.630,42004689,42001455,1484034200468942001455,152.530438,17a,152.530490,0,1,LINESTRING (-88.02748423776256 47.473549142674...
3,4,A21,M 26,11449,11467,11448,11466,49950,49950,,...,2.535,42001466,42004689,1484034200146642004689,66.213698,17a,66.213653,0,1,LINESTRING (-88.02729213151295 47.472967779585...
4,5,A41,Middle Point Rd,11448,11000,11449,11001,49950,49950,,...,0.872,42001466,42001470,34205864200146642001470,1402.925854,17a,1402.926114,0,0,LINESTRING (-88.02729213151295 47.472967779585...


(PR,BPT,EPT) columns are primary keys that identify a unique road segment. Let's first check if there's any duplicate segments.

In [42]:
keys = ['PR', 'BPT', 'EPT']
True in roads.duplicated(keys).values

True

We can drop the duplicate road and keep the latest.

In [43]:
roads_dropdupe = roads.drop_duplicates(subset=keys)
len(roads_dropdupe)

794907

Then let's read in road summary data to decide which roads are still in use.

In [44]:
file2 = '2016_master_table_id.txt'
ids = gp.pd.read_csv(file2)
ids.head()

,ID,PR,BPT,EPT,YEAR_ADDED,YEAR_REMOVED
0,2004000001,0,42000023,42000018,2004,2005.0
1,2004000002,0,42000151,42000128,2004,2005.0
2,2004000003,0,42000151,42000161,2004,2005.0
3,2004000004,0,42000170,42000148,2004,2005.0
4,2004000005,0,42000158,42000170,2004,2005.0


Active roads have null values in YEAR_REMOVED column.

In [46]:
idx = ids['YEAR_REMOVED'].isnull()
len(ids[idx])

794178

Next we can inner merge two dataframes on primary keys to get geometry feature of the active roads.

In [50]:
roads_dropdupe[['BPT','EPT']] = roads_dropdupe[['BPT','EPT']].astype('int')
active_roads = ids[idx].merge(roads_dropdupe, how='inner', left_on=['PR','BPT','EPT'],right_on=['PR','BPT','EPT'])
len(active_roads`)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


789096

In [61]:
active_roads.head()

,ID,PR,BPT,EPT,YEAR_ADDED,YEAR_REMOVED,OBJECTID,FCC,RDNAME,FRADDL,...,BMP,EMP,LRS_LINK,LENGTH,VER,ShapeSTLength,ADJ_CNTY,LEGALSYSTEM,geometry,ids
0,2004000199,3420585,42001455,42001458,2004,NaN,1,A41,North Point Rd,11448,...,0.000,0.835,34205854200145542001458,1343.174725,17a,1343.174920,0,0,LINESTRING (-88.02735862891302 47.474893562611...,"(245263, 245264)"
1,2004000201,3420586,42001466,42001470,2004,NaN,5,A41,Middle Point Rd,11448,...,0.000,0.872,34205864200146642001470,1402.925854,17a,1402.926114,0,0,LINESTRING (-88.02729213151295 47.472967779585...,"(245265, 245266)"
2,2004000202,3420658,42004435,42004436,2004,NaN,6,A41,Harbor Coast Ln,14041,...,0.142,0.505,34206584200443542004436,583.158955,17a,583.158926,0,0,LINESTRING (-87.91845890770243 47.474059047116...,"(246128, 246129)"
3,2004000204,3420658,42004433,42004435,2004,NaN,8,A41,Harbor Lake Estates 3,9421,...,0.000,0.142,34206584200443342004435,229.198743,17a,229.198843,0,0,LINESTRING (-87.91840197064076 47.472014531962...,"(246126, 246128)"
4,2004000205,148403,42001490,42004433,2004,NaN,9,A21,M 26,14001,...,8.064,8.134,1484034200149042004433,112.714170,17a,112.714166,0,1,LINESTRING (-87.91988864667995 47.471907516301...,"(245268, 246126)"


## Generate network

First let's prepare the nodes of the network which can be generated by combining beginning and ending points.

In [51]:
nodeset1 = set(active_roads['BPT'])
nodeset2 = set(active_roads['EPT'])
nodeset = list(nodeset1.union(nodeset2))
nodeset.sort()

Now that we have all the nodes, we can prepare the edges of the network. Each edge(road) is uniquely defined by its source and target node indexes.

In [55]:
ID = dict(zip(nodeset,range(len(nodeset))))
ID_inv = dict(zip(range(len(nodeset)),nodeset))
active_roads['ids'] = active_roads.apply(lambda df: (ID[df.at['BPT']],ID[df.at['EPT']]), axis=1)
active_roads['ids'].head()

0    (245263, 245264)
1    (245265, 245266)
2    (246128, 246129)
3    (246126, 246128)
4    (245268, 246126)
Name: ids, dtype: object

Add nodes as vertices. 

In [38]:
g = ig.Graph(directed=False)
g.add_vertices(len(nodeset))
g.vs['nodeid'] = [ID_inv[x] for x in range(len(nodeset))]

'IGRAPH U-W- 613878 789096 -- \n+ attr: nodeid (v), id (e), weight (e)'

Time to add the edges. Edges are specified by pairs of integers, so [(0,1), (1,2)] denotes a list of two edges: one between the first and the second, and the other one between the second and the third vertices of the graph. Passing the node pairs generated in the last step to Graph.add_edges() adds edges to the road network:

In [75]:
g.add_edges(active_roads['ids'].tolist())
g.es['weight'] = active_roads['LENGTH']
g.es['id'] = active_roads['ID']
g.es['name'] = active_roads['RDNAME']
g.summary()

'IGRAPH U-W- 613878 3945480 -- \n+ attr: nodeid (v), id (e), name (e), weight (e)'

Take a look at a random edge, which has the attributes weight(road length), id (road id) and name(road name).

In [117]:
g.es[5000]

igraph.Edge(<igraph.Graph object at 0x115a259a8>, 5000, {'weight': 135.48225077999999, 'id': 2004005956, 'name': 'M 26'})